In [5]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from pathlib import Path
from tqdm import tqdm

sys.path.append(str(Path("..").resolve()))
from src import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Migrate MySQL to MongoDB
As the database schema has evolved significantly since the current MySQL database was created, we first migrate a subset of the collections using [_MongoDB Relational Migrator_](https://www.mongodb.com/resources/solutions/use-cases/mysql-to-mongodb). We then complete the migration manually using handwritten queries.

In [8]:
connector=MongoDBConnector("museo_omero")
db=connector.db

In [9]:
for collection in connector.collections:
    schema = connector.infer_schema(collection)
    cprint(
        f"[{collection}]:",
        *[f"rand:{_}" for _ in schema],
    )

[biglietti]: _id costo data_stampa evento gruppo id
[reparti]: _id nome piano posti_occ posti_tot stanza
[dipendenti]: _id cellulare cognome curriculum data_nasc data_registrazione email id luogo_nasc nome sesso
[opere]: _id data descrizione id sala tipologia
[visitor]: _id cellulare cognome email id name  tariffa tessera
[laboratori]: _id costo_classe costo_persona durata nome tipologia
[questionari]: _id accompagnatori_visita data_compilazione motivazione_visita numero_visite ritorno tipologia_visita titolo_studi valutazione_esperienza valutazione_struttura valutazione_visita visitatore
[artisti]: _id cognome data_nasc id luogo_nasc nome sesso


In [4]:
db.clienti.rename("visitor")
db.visitor.update_many({}, {"$unset": {"indirizzo": ""}})
db.visitor.update_many({}, {"$rename": {"nome": "name "}})

UpdateResult({'n': 101, 'nModified': 17, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [5]:
result = db.clienti.find({})

for document in result:
    print(document)

{'_id': ObjectId('68b572f8845fcdbabc1c891f'), 'id': 2, 'nome': 'Mario', 'cognome': 'Orsini', 'tessera': 0, 'cellulare': '+39 (33) 0226 2', 'email': 'heaven.kemmer@gmail.com'}
{'_id': ObjectId('68b572f8845fcdbabc1c8920'), 'id': 6, 'nome': 'Giovanni', 'cognome': 'Casini', 'tessera': 0, 'cellulare': '+39 (33) 5875 7', 'email': 'ureinger@white.com'}
{'_id': ObjectId('68b572f8845fcdbabc1c8921'), 'id': 10, 'nome': 'Luigi', 'cognome': 'Re', 'tessera': 0, 'cellulare': '+39 (33) 0474 2', 'email': 'jane63@hotmail.com'}
{'_id': ObjectId('68b572f8845fcdbabc1c8922'), 'id': 16, 'nome': 'Filippo', 'cognome': 'Capriotti', 'tessera': 1, 'cellulare': '+39 (33) 8764 9', 'email': 'roger.murray@gmail.com'}
{'_id': ObjectId('68b572f8845fcdbabc1c8923'), 'id': 25, 'nome': 'Marta', 'cognome': "D'Ambrosio", 'tessera': 1, 'cellulare': '+39 (33) 0122 2', 'email': 'larson.mafalda@gmail.com'}
{'_id': ObjectId('68b572f8845fcdbabc1c8924'), 'id': 28, 'nome': 'Casilde', 'cognome': 'Tambone', 'tessera': 1, 'cellulare': 